# Notebook de mejora

Este notebook es un ejemplo cómo se puede mejorar los hiperparámetros de un algoritmo, y medir la importancia de atributos https://www.kaggle.com/c/house-prices-advanced-regression-techniques/ de la práctica 3 de Inteligencia de Negocio. 

Incluyo el código del notebook anterior, no se explicarán.

Es un ejemplo que es claramente mejorable, se deja a el/la estudiante el mejorarlo para obtener mejores resultados.

In [10]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer


In [11]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv", na_values="NaN") # Definimos na_values para identificar bien los valores perdidos


if 'Id' in train:
    train.drop('Id', axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

test = pd.read_csv("test.csv", na_values="NaN")
test_ids = test.Id
test = test.drop('Id', axis=1)

input_all = pd.concat([train.drop('SalePrice', axis=1), test])

col_cat = list(input_all.select_dtypes(exclude=np.number).columns)

from sklearn.impute import SimpleImputer

# Valores categóricos por el más frecuente
imputer_cat = SimpleImputer(strategy="most_frequent")
imputer_cat.fit(input_all[col_cat])
train[col_cat] = imputer_cat.transform(train[col_cat])
test[col_cat] = imputer_cat.transform(test[col_cat])

# Valores numéricos por KNN
col_num = list(train.select_dtypes(include=np.number).columns)
col_num.remove('SalePrice')

# Configuración del imputador KNN
knn_imputer = KNNImputer(n_neighbors=5)  # Puedes ajustar el número de vecinos según tus necesidades

# Imputación para el conjunto de entrenamiento
train[col_num] = knn_imputer.fit_transform(train[col_num])

# Imputación para el conjunto de prueba
test[col_num] = knn_imputer.transform(test[col_num])

from sklearn.preprocessing import LabelEncoder
labelers = {}
test_l = test.copy()
train_l = train.copy()


y_train = train_l.SalePrice
X_train = train_l.drop('SalePrice', axis=1)

if 'Id' in test_l:
    test_l.drop('Id', axis=1, inplace=True)

X_test = test_l

In [12]:
X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60.0,RL,65.0,8450.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,2.0,2008.0,WD,Normal
1,20.0,RL,80.0,9600.0,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,0.0,0.0,Ex,MnPrv,Shed,0.0,5.0,2007.0,WD,Normal
2,60.0,RL,68.0,11250.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,9.0,2008.0,WD,Normal
3,70.0,RL,60.0,9550.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0.0,0.0,Ex,MnPrv,Shed,0.0,2.0,2006.0,WD,Abnorml
4,60.0,RL,84.0,14260.0,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,0.0,0.0,Ex,MnPrv,Shed,0.0,12.0,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,8.0,2007.0,WD,Normal
1456,20.0,RL,85.0,13175.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,2.0,2010.0,WD,Normal
1457,70.0,RL,66.0,9042.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,GdPrv,Shed,2500.0,5.0,2010.0,WD,Normal
1458,20.0,RL,68.0,9717.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,4.0,2010.0,WD,Normal


In [13]:
y_train

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [14]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_log_error

# Split the data into training and validation sets
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a column transformer to handle categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Transform the training data
X_train2_transformed = preprocessor.fit_transform(X_train2)
X_train_transformed = preprocessor.fit_transform(X_train)

# Create an XGBoost regressor
xg_reg = xgb.XGBRegressor(objective='reg:squaredlogerror', seed=42)

# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(xg_reg, X_train2_transformed, y_train2, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores))
print("Std CV Score: ", np.std(cv_scores))

# Fit the model on the entire training set
xg_reg.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = xg_reg.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score)


Cross-validation scores:  [7.46938917 7.44492932 7.41372314 7.46061306 7.47845455 7.46462243
 7.46449019 7.42516991 7.4336704  7.48145291]
Mean CV Score:  7.4536515092289
Std CV Score:  0.02191547665096053
Validation Score (neg_mean_squared_log_error):  52.2259646078938


In [15]:
# Transform the training data
X_train_transformed = preprocessor.fit_transform(X_train)

# Create an XGBoost regressor
xg_reg = xgb.XGBRegressor(colsample_bytree=0.5, gamma=0.05, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.5, n_estimators=2200,
                             reg_alpha=0.5, reg_lambda=0.8,
                             subsample=0.5,
                             random_state =42, seed=42)

# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(xg_reg, X_train_transformed, y_train, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores))
print("Std CV Score: ", np.std(cv_scores))

# Fit the model on the entire training set
xg_reg.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = xg_reg.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score)

Cross-validation scores:  [0.11244047 0.16100895 0.1078119  0.12312627 0.16104299 0.1504215
 0.13337552 0.1056039  0.12345604 0.08130693]
Mean CV Score:  0.12595944729489833
Std CV Score:  0.024591254579935728
Validation Score (neg_mean_squared_log_error):  0.0008367440039281323


In [16]:
# Fit the model on the entire training set
xg_reg.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = xg_reg.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score)

X_train_final = X_train.copy()

# Add a new column to the training set with predicted values
X_train_final["PredictedValuesXGBoost"] = xg_reg.predict(X_train_transformed)
X_train_final["ValidScoreXGBoost"] = validation_score
X_train_final["MeanCVXGBoost"] = np.mean(cv_scores)
X_train_final["StdCVXGBoost"] = np.std(cv_scores)

# Print or store the results
results_df = pd.DataFrame({
    'ActualValues': y_train,
    'PredictedValuesXGBoost': X_train_final["PredictedValuesXGBoost"]
})

# Optionally, you can also include the validation set results
results_valid_df = pd.DataFrame({
    'ActualValues_valid': y_valid,
    'PredictedValues_valid': y_pred_valid
})

# Print or display the results
print("Training Set Predictions:")
print(results_df.head())
X_train_final


Validation Score (neg_mean_squared_log_error):  0.0008367440039281323
Training Set Predictions:
   ActualValues  PredictedValuesXGBoost
0        208500           208661.250000
1        181500           180975.921875
2        223500           221752.593750
3        140000           141739.375000
4        250000           257578.953125


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,PredictedValuesXGBoost,ValidScoreXGBoost,MeanCVXGBoost,StdCVXGBoost
0,60.0,RL,65.0,8450.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,0.0,2.0,2008.0,WD,Normal,208661.250000,0.000837,0.125959,0.024591
1,20.0,RL,80.0,9600.0,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,Shed,0.0,5.0,2007.0,WD,Normal,180975.921875,0.000837,0.125959,0.024591
2,60.0,RL,68.0,11250.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,Shed,0.0,9.0,2008.0,WD,Normal,221752.593750,0.000837,0.125959,0.024591
3,70.0,RL,60.0,9550.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,Shed,0.0,2.0,2006.0,WD,Abnorml,141739.375000,0.000837,0.125959,0.024591
4,60.0,RL,84.0,14260.0,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,Shed,0.0,12.0,2008.0,WD,Normal,257578.953125,0.000837,0.125959,0.024591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,0.0,8.0,2007.0,WD,Normal,172345.281250,0.000837,0.125959,0.024591
1456,20.0,RL,85.0,13175.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,0.0,2.0,2010.0,WD,Normal,206520.234375,0.000837,0.125959,0.024591
1457,70.0,RL,66.0,9042.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,2500.0,5.0,2010.0,WD,Normal,264303.093750,0.000837,0.125959,0.024591
1458,20.0,RL,68.0,9717.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,0.0,4.0,2010.0,WD,Normal,141166.765625,0.000837,0.125959,0.024591


In [17]:
import lightgbm as lgb

# Split the data into training and validation sets
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a column transformer to handle categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Transform the training data
X_train2_transformed = preprocessor.fit_transform(X_train2)
X_train_transformed = preprocessor.fit_transform(X_train)


model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=1000,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.25,
                              feature_fraction_seed=9, bagging_seed=9, seed=42)

# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(model_lgb, X_train2_transformed, y_train2, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores))
print("Std CV Score: ", np.std(cv_scores))

# Fit the model on the entire training set
model_lgb.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = model_lgb.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score_lgb = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score_lgb)

[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002390 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1324
[LightGBM] [Info] Number of data points in the train set: 1051, number of used features: 174
[LightGBM

[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_f

In [18]:
# Fit the model on the entire training set
model_lgb.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = model_lgb.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score_lgb = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score_lgb)

# Add a new column to the training set with predicted values
X_train_final["PredictedValuesLightGBM"] = model_lgb.predict(X_train_transformed)
X_train_final["ValidScoreLightGBM"] = validation_score_lgb
X_train_final["MeanCVLightGBM"] = np.mean(cv_scores)
X_train_final["StdCVLightGBM"] = np.std(cv_scores)


# Print or store the results
results_df = pd.DataFrame({
    'ActualValues': y_train,
    'PredictedValuesLightGBM': X_train_final["PredictedValuesLightGBM"]
})

# Optionally, you can also include the validation set results
results_valid_df = pd.DataFrame({
    'ActualValues_valid': y_valid,
    'PredictedValues_valid': y_pred_valid
})

# Print or display the results
print("Training Set Predictions:")
print(results_df.head())
X_train_final

[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1396
[LightGBM] [Info] Number of data point

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,SaleType,SaleCondition,PredictedValuesXGBoost,ValidScoreXGBoost,MeanCVXGBoost,StdCVXGBoost,PredictedValuesLightGBM,ValidScoreLightGBM,MeanCVLightGBM,StdCVLightGBM
0,60.0,RL,65.0,8450.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,208661.250000,0.000837,0.125959,0.024591,201412.688312,0.009167,0.122299,0.020318
1,20.0,RL,80.0,9600.0,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,WD,Normal,180975.921875,0.000837,0.125959,0.024591,172649.796284,0.009167,0.122299,0.020318
2,60.0,RL,68.0,11250.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,WD,Normal,221752.593750,0.000837,0.125959,0.024591,216089.583937,0.009167,0.122299,0.020318
3,70.0,RL,60.0,9550.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,WD,Abnorml,141739.375000,0.000837,0.125959,0.024591,164844.080361,0.009167,0.122299,0.020318
4,60.0,RL,84.0,14260.0,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,WD,Normal,257578.953125,0.000837,0.125959,0.024591,286599.782010,0.009167,0.122299,0.020318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,172345.281250,0.000837,0.125959,0.024591,171617.835704,0.009167,0.122299,0.020318
1456,20.0,RL,85.0,13175.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,206520.234375,0.000837,0.125959,0.024591,202143.774679,0.009167,0.122299,0.020318
1457,70.0,RL,66.0,9042.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,264303.093750,0.000837,0.125959,0.024591,259073.568094,0.009167,0.122299,0.020318
1458,20.0,RL,68.0,9717.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,141166.765625,0.000837,0.125959,0.024591,138716.279058,0.009167,0.122299,0.020318


In [25]:
# Split the data into training and validation sets
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a column transformer to handle categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Transform the training data
X_train2_transformed = preprocessor.fit_transform(X_train2)
X_train_transformed = preprocessor.fit_transform(X_train)


random_forest = RandomForestRegressor(criterion='squared_error', max_depth=20, random_state=42)

# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(random_forest, X_train2_transformed, y_train2, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores))
print("Std CV Score: ", np.std(cv_scores))

# Fit the model on the entire training set
random_forest.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = random_forest.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score_rf = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score_rf)

Cross-validation scores:  [0.14830938 0.14322594 0.13947216 0.15327671 0.14407841 0.20182229
 0.12324858 0.13073712 0.11916472 0.1321026 ]
Mean CV Score:  0.14354379127798272
Std CV Score:  0.021981664189249215
Validation Score (neg_mean_squared_log_error):  0.0037508865993548493


In [26]:
# Fit the model on the entire training set
random_forest.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = random_forest.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score_rf = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score_rf)

# Add a new column to the training set with predicted values
X_train_final["PredictedValuesRandomForest"] = random_forest.predict(X_train_transformed)
X_train_final["ValidScoreRandomForest"] = validation_score_rf
X_train_final["MeanCVRandomForest"] = np.mean(cv_scores)
X_train_final["StdCVRandomForest"] = np.std(cv_scores)


X_train_final

Validation Score (neg_mean_squared_log_error):  0.0037508865993548493


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MeanCVXGBoost,StdCVXGBoost,PredictedValuesLightGBM,ValidScoreLightGBM,MeanCVLightGBM,StdCVLightGBM,PredictedValuesRandomForest,ValidScoreRandomForest,MeanCVRandomForest,StdCVRandomForest
0,60.0,RL,65.0,8450.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.125959,0.024591,201412.688312,0.009167,0.122299,0.020318,205523.466667,0.003751,0.143544,0.021982
1,20.0,RL,80.0,9600.0,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,0.125959,0.024591,172649.796284,0.009167,0.122299,0.020318,175070.320000,0.003751,0.143544,0.021982
2,60.0,RL,68.0,11250.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0.125959,0.024591,216089.583937,0.009167,0.122299,0.020318,220379.983333,0.003751,0.143544,0.021982
3,70.0,RL,60.0,9550.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0.125959,0.024591,164844.080361,0.009167,0.122299,0.020318,159469.780000,0.003751,0.143544,0.021982
4,60.0,RL,84.0,14260.0,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,0.125959,0.024591,286599.782010,0.009167,0.122299,0.020318,265255.930000,0.003751,0.143544,0.021982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.125959,0.024591,171617.835704,0.009167,0.122299,0.020318,176388.504786,0.003751,0.143544,0.021982
1456,20.0,RL,85.0,13175.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.125959,0.024591,202143.774679,0.009167,0.122299,0.020318,206899.500000,0.003751,0.143544,0.021982
1457,70.0,RL,66.0,9042.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.125959,0.024591,259073.568094,0.009167,0.122299,0.020318,258720.270000,0.003751,0.143544,0.021982
1458,20.0,RL,68.0,9717.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.125959,0.024591,138716.279058,0.009167,0.122299,0.020318,143048.815833,0.003751,0.143544,0.021982


In [59]:
for col in categorical_cols:
    # Find the most frequent value in X_train for the current column
    most_frequent_value = X_train[col].mode()[0]

    # Replace values in X_test with the most frequent value if not in X_train
    X_test[col] = X_test[col].apply(lambda x: x if x in X_train[col].unique() else most_frequent_value)
    
X_test_transformed = preprocessor.fit_transform(X_test)


In [61]:
X_test_transformed

<1459x270 sparse matrix of type '<class 'numpy.float64'>'
	with 97930 stored elements in Compressed Sparse Row format>

In [63]:
len(X_train_transformed[0])


298

In [72]:
# Apply one-hot encoding to the test set
X_test_transformed = pd.get_dummies(X_test, columns=categorical_cols)
X_train_transformed = pd.get_dummies(X_train, columns=categorical_cols)

# Ensure the same columns are present in both X_train_transformed and X_test_transformed
common_columns = X_train_transformed.columns.intersection(X_test_transformed.columns)
X_test_transformed = X_test_transformed[common_columns]

In [77]:
# Find missing columns in X_test_transformed
missing_columns = set(X_train_transformed.columns) - set(X_test_transformed.columns)

# Add missing columns to X_test_transformed and fill with zeros
for col in missing_columns:
    X_test_transformed[col] = 0
    
# Ensure the same columns are present in both X_train_transformed and X_test_transformed
common_columns = X_train_transformed.columns.intersection(X_test_transformed.columns)

# Get the feature names from the training set
feature_names = X_train_transformed[common_columns].columns

# Reorder columns in the test set to match the order of feature names from the training set
X_test_transformed = X_test_transformed[feature_names]

In [78]:
X_test_transformed

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20.0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,144.0,...,0,0,0,1,0,0,0,0,1,0
1,20.0,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60.0,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,60.0,78.0,9978.0,6.0,6.0,1998.0,1998.0,20.0,602.0,0.0,...,0,0,0,1,0,0,0,0,1,0
4,120.0,43.0,5005.0,8.0,5.0,1992.0,1992.0,0.0,263.0,0.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160.0,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1455,160.0,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,0.0,...,0,0,0,1,1,0,0,0,0,0
1456,20.0,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,0.0,...,0,0,0,1,1,0,0,0,0,0
1457,85.0,62.0,10441.0,5.0,5.0,1992.0,1992.0,0.0,337.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [79]:
X_train_transformed

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,0,0,0,1,1,0,0,0,0,0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1456,20.0,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,163.0,...,0,0,0,1,0,0,0,0,1,0
1457,70.0,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,275.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1458,20.0,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,1029.0,...,0,0,0,1,0,0,0,0,1,0


In [82]:
from sklearn.linear_model import LinearRegression

random_forest.fit(X_train_transformed, y_train)
xg_reg.fit(X_train_transformed, y_train)
model_lgb.fit(X_train_transformed, y_train)

# Step 3: Generate Meta-Features
meta_features_train = np.column_stack([
    random_forest.predict(X_train_transformed),
    xg_reg.predict(X_train_transformed),
    model_lgb.predict(X_train_transformed)
])

meta_features_test = np.column_stack([
    random_forest.predict(X_test_transformed),
    xg_reg.predict(X_test_transformed),
    model_lgb.predict(X_test_transformed)
])

# Combine original features with meta-features
X_train_stacked = np.column_stack([X_train_transformed, meta_features_train])
X_test_stacked = np.column_stack([X_test_transformed, meta_features_test])

# Step 4: Train the Final Estimator
final_estimator = LinearRegression()
final_estimator.fit(X_train_stacked, y_train)

# Step 5: Make Predictions on Test Data
final_predictions = final_estimator.predict(X_test_stacked)

final_predictions

[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001130 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise

array([125467.98128198, 154924.98331736, 180181.81391571, ...,
       162459.69949339, 114722.68432425, 207955.97847746])

In [83]:
salida = pd.DataFrame({'Id': test_ids, 'SalePrice': final_predictions})
salida.to_csv("Prueba8.csv", index=False)